In [ ]:
# 자동 음성 진단 시스템: Praat 분석 결과 기반
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

# 1. 자동 인코딩 감지 및 CSV 로드 함수
def load_praat_csv_via_upload():
    print("📂 Praat 분석 CSV 파일을 업로드하세요")
    uploaded = files.upload()
    for filename in uploaded.keys():
        # 시도할 인코딩 리스트
        encodings_to_try = ['utf-8', 'utf-16', 'cp949', 'euc-kr']
        for enc in encodings_to_try:
            try:
                df = pd.read_csv(filename, encoding=enc)
                print(f"✅ 파일 로드 성공: {filename} (인코딩: {enc})")
                return df
            except Exception as e:
                continue
        raise ValueError("❌ CSV 파일 로딩 실패: 인코딩 문제")

# 2. 피치 대역 구분 함수
def classify_pitch_band(pitch):
    if pitch < 220:
        return "저음 (Low)"
    elif pitch < 350:
        return "중음 (Mid)"
    else:
        return "고음 (High)"

# 3. 진단 기준 함수
def diagnose_vocal(row):
    pitch = row['Mean Pitch (Hz)']
    f1 = row['Mean F1 (Hz)']
    f2 = row['Mean F2 (Hz)']
    intensity = row['Mean Intensity (dB)']
    filename = row['Filename']

    band = classify_pitch_band(pitch)
    feedback = [f"[{filename}] ({band})"]

    # Pitch 기준
    if pitch < 220:
        feedback.append(f"- Pitch {pitch:.1f}Hz: ⚠ 너무 낮음")
    elif pitch >= 350:
        feedback.append(f"- Pitch {pitch:.1f}Hz: ⚠ 너무 높음")
    else:
        feedback.append(f"- Pitch {pitch:.1f}Hz: ✅ 정상")

    # F1 기준
    if f1 < 250:
        feedback.append(f"- F1 {f1:.1f}Hz: ⚠ 과도한 닫힘 (공명 부족)")
    elif f1 > 600:
        feedback.append(f"- F1 {f1:.1f}Hz: ⚠ 과도한 개방 (불안정)")
    else:
        feedback.append(f"- F1 {f1:.1f}Hz: ✅ 안정적")

    # F2 기준
    if f2 < 1200:
        feedback.append(f"- F2 {f2:.1f}Hz: ⚠ 발음 명료도 저하 가능")
    elif f2 > 2500:
        feedback.append(f"- F2 {f2:.1f}Hz: ⚠ 과도한 전설화 (혀 위치 불균형)")
    else:
        feedback.append(f"- F2 {f2:.1f}Hz: ✅ 명료")

    # Intensity 기준
    if intensity < 60:
        feedback.append(f"- Intensity {intensity:.1f}dB: ⚠ 에너지 부족")
    elif intensity > 85:
        feedback.append(f"- Intensity {intensity:.1f}dB: ⚠ 발성 강도 과도 (과긴장 의심)")
    else:
        feedback.append(f"- Intensity {intensity:.1f}dB: ✅ 양호")

    return "\n".join(feedback)

# 4. 문제 분류 함수
def classify_issues(row):
    issues = []
    if row['Mean Pitch (Hz)'] < 220:
        issues.append('Low Pitch')
    elif row['Mean Pitch (Hz)'] > 350:
        issues.append('High Pitch')

    if row['Mean F1 (Hz)'] < 250:
        issues.append('Low F1')
    elif row['Mean F1 (Hz)'] > 600:
        issues.append('High F1')

    if row['Mean F2 (Hz)'] < 1200:
        issues.append('Low F2')
    elif row['Mean F2 (Hz)'] > 2500:
        issues.append('High F2')

    if row['Mean Intensity (dB)'] < 60:
        issues.append('Low Intensity')
    elif row['Mean Intensity (dB)'] > 85:
        issues.append('High Intensity')

    return issues

# 5. 시각화 함수
def plot_metric_distributions(df):
    df['Trial'] = df['Filename'].str.extract(r'(t\d)')
    metrics = ['Mean Pitch (Hz)', 'Mean F1 (Hz)', 'Mean F2 (Hz)', 'Mean Intensity (dB)']
    plt.figure(figsize=(16, 10))
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 2, i)
        sns.boxplot(data=df, x='Trial', y=metric, palette='pastel')
        plt.title(f'{metric} by Trial')
        plt.xlabel('Trial')
        plt.ylabel(metric)
    plt.tight_layout()
    plt.show()

# 6. 통계 요약 함수
def summarize_issues(df):
    from collections import Counter
    df['Issues'] = df.apply(classify_issues, axis=1)
    all_issues = sum(df['Issues'], [])
    issue_counts = Counter(all_issues)
    return pd.DataFrame(issue_counts.items(), columns=['Issue Type', 'Count']).sort_values(by='Count', ascending=False)

# 7. 전체 실행 함수
def run_diagnosis_via_upload():
    df = load_praat_csv_via_upload()
    # 수치형 변환
    df['Mean Pitch (Hz)'] = pd.to_numeric(df['Mean Pitch (Hz)'], errors='coerce')
    df['Mean F1 (Hz)'] = pd.to_numeric(df['Mean F1 (Hz)'], errors='coerce')
    df['Mean F2 (Hz)'] = pd.to_numeric(df['Mean F2 (Hz)'], errors='coerce')
    df['Mean Intensity (dB)'] = pd.to_numeric(df['Mean Intensity (dB)'], errors='coerce')

    print("\n========== 개별 음적 진단 ==========")
    for result in df.apply(diagnose_vocal, axis=1):
        print(result)
        print()

    print("\n========== 시각화 ==========")
    plot_metric_distributions(df)

    print("\n========== 문제 요약 통계 ==========")
    summary = summarize_issues(df)
    print(summary)

# ✅ 실행
run_diagnosis_via_upload()
